In [11]:
import libaarhusxyz
import downfile
import os.path
import pandas as pd
import numpy as np
import re

In [2]:
basepath = "../examples-secret/oldwb"

In [3]:
old = {}
new = {}
for pathxyz in os.listdir(basepath):
    if not pathxyz.endswith("xyz"): continue
    pathxyz = os.path.join(basepath, pathxyz)
    print(pathxyz)
    new[pathxyz] = libaarhusxyz.XYZ(pathxyz).model_dict
    pathdown = os.path.splitext(pathxyz)[0] + ".orig.down"
    old[pathxyz] = downfile.parse(pathdown)

../examples-secret/oldwb/NVE20210916_1_AVG_export_example_averagde_data_export_oldWB.xyz
../examples-secret/oldwb/SCI_1_Pro3_MOD_inv_example_SCI_inversion_export_oldWB.xyz
../examples-secret/oldwb/SCI_1_Pro3_MOD_dat_example_SCI_inversion_export_oldWB.xyz
../examples-secret/oldwb/NVE20210916_1_RAW_export_example_averagde_data_export_oldWB.xyz
../examples-secret/oldwb/SCI_1_Pro3_MOD_syn_example_SCI_inversion_export_oldWB.xyz


In [4]:
class Difference(object):
    def __init__(self, path, diff, a, b):
        self.path = path
        self.diff = diff
        self.a = a
        self.b = b
    def __str__(self):
        return "%s: %s %s %s" % (".".join([str(i) for i in self.path]), self.a, self.diff, self.b)
        
def compare(a, b, path= []):
    at = type(a)
    bt = type(b)
    if at is not bt:
        yield Difference(path, "not same type as", at, bt); return
    
    if isinstance(a, dict):
        ak = set(a.keys())
        bk = set(b.keys())
        mbk = ak - bk
        mak = bk - ak
        if mbk:
            yield Difference(path, "missing in", mbk, "b")
        if mak:
            yield Difference(path, "missing in", mak, "a")            
        for key in ak.intersection(bk):
            for diff in compare(a[key], b[key], path + [key]):
                yield diff
    elif isinstance(a, list):
        if len(a) != len(b):
            yield Difference(path, "not same length", len(a), len(b))
        for idx, (ai, bi) in enumerate(zip(a, b)):
            for diff in compare(ai, bi, path + [idx]):
                yield diff
    elif isinstance(a, (pd.Index, pd.Series)):
        if len(a) != len(b):
            yield Difference(path, "not same length", len(a), len(b))
        sa = set(a)
        sb = set(b)
        msb = sa - sb
        msa = sb - sa
        if msb:
            yield Difference(path, "missing in", msb, "b")
        if msa:
            yield Difference(path, "missing in", msa, "a")     
    elif isinstance(a, pd.DataFrame):
        for diff in compare(a.columns, b.columns, path+["columns"]):
            yield diff
        for diff in compare(a.index, b.index, path+["columns"]):
            yield diff
        for col in set(a.columns).intersection(b.columns):
            merged = a.loc[
                a.index.isin(b.index), [col]
            ].rename(
                columns={col: "a"}
            ).join(
                b.loc[b.index.isin(a.index), [col]].rename(
                columns={col: "b"}
            ))
            merged = merged.loc[merged.a != merged.b]
            
            for diff in compare(merged.a, merged.b, path + [col]):
                yield diff
    else:
        if a != b:
            yield Difference(path, "!=", a, b)    

In [5]:
for key in old.keys():
    print(key)
    for idx, diff in enumerate(compare(old[key], new[key])):
        print("     " + str(diff))
        if idx > 100:
            print("Too many errors...")
            break


../examples-secret/oldwb/NVE20210916_1_AVG_export_example_averagde_data_export_oldWB.xyz
../examples-secret/oldwb/SCI_1_Pro3_MOD_inv_example_SCI_inversion_export_oldWB.xyz
../examples-secret/oldwb/SCI_1_Pro3_MOD_dat_example_SCI_inversion_export_oldWB.xyz
../examples-secret/oldwb/NVE20210916_1_RAW_export_example_averagde_data_export_oldWB.xyz
../examples-secret/oldwb/SCI_1_Pro3_MOD_syn_example_SCI_inversion_export_oldWB.xyz


In [7]:
old.keys()

dict_keys(['../examples-secret/oldwb/NVE20210916_1_AVG_export_example_averagde_data_export_oldWB.xyz', '../examples-secret/oldwb/SCI_1_Pro3_MOD_inv_example_SCI_inversion_export_oldWB.xyz', '../examples-secret/oldwb/SCI_1_Pro3_MOD_dat_example_SCI_inversion_export_oldWB.xyz', '../examples-secret/oldwb/NVE20210916_1_RAW_export_example_averagde_data_export_oldWB.xyz', '../examples-secret/oldwb/SCI_1_Pro3_MOD_syn_example_SCI_inversion_export_oldWB.xyz'])

In [9]:
old['../examples-secret/oldwb/NVE20210916_1_AVG_export_example_averagde_data_export_oldWB.xyz']["flightlines"].index

RangeIndex(start=0, stop=10023, step=1)

In [10]:
new['../examples-secret/oldwb/NVE20210916_1_AVG_export_example_averagde_data_export_oldWB.xyz']["flightlines"].index

Int64Index([    0,     1,     2,     3,     4,     5,     6,     7,     8,
                9,
            ...
            10013, 10014, 10015, 10016, 10017, 10018, 10019, 10020, 10021,
            10022],
           dtype='int64', length=10023)

In [5]:
# x =  pd.DataFrame({"a": [1, 2, 3.4, "hej", "mumu", "hehe"]})
x =  pd.DataFrame({"a": [1, 2, 3.4]})


In [6]:
x.a.astype("O").str.match(r"^he.*")

AttributeError: Can only use .str accessor with string values!

In [5]:
basedir = "../examples-secret/newclean"

In [31]:
for dirpath, dirnames, filenames in os.walk(basedir):
    for name in filenames:
        if not name.endswith("xyz"): continue

        path = os.path.join(dirpath, name)
        print(path)
        with open(path) as f:
            d = f.read()

        o = []
        last = 0
        for m in re.finditer(r"(\s|^)(?P<number>[0-9+-]\.[0-9E+-]+)($|\s)", d):
            s = m.start("number")
            o.append(d[last:s])
            o.append(('%.3E' % (np.random.random() * 2 * float(m["number"]))).upper())
            last = m.end("number")
        o.append(d[last:])
        o = "".join(o)
        
        o = re.sub(r"[A-Z]:\\[^\s]+", r"G:\\Test - 1", o)

        with open(path, "w") as f:
            f.write(o)


../examples-secret/newclean/aarhus_workbench.5.9.4.0/1116_1_pro_AVG_export.xyz
../examples-secret/newclean/aarhus_workbench.5.9.4.0/1116_1_pro_I08_MOD_dat.xyz
../examples-secret/newclean/aarhus_workbench.5.9.4.0/1116_1_pro_I08_MOD_inv.xyz
../examples-secret/newclean/aarhus_workbench.5.9.4.0/1116_1_pro_I08_MOD_syn.xyz
../examples-secret/newclean/aarhus_workbench.6.2.1.0/02_2_AVG_export.xyz
../examples-secret/newclean/aarhus_workbench.6.0.1.1/CL_P20161207_SRM_UTM12_MOD_dat.xyz
../examples-secret/newclean/aarhus_workbench.6.0.1.1/CL_P20161207_SRM_UTM12_MOD_inv.xyz
../examples-secret/newclean/aarhus_workbench.6.0.1.1/CL_P20161207_SRM_UTM12_MOD_syn.xyz
../examples-secret/newclean/aarhus_workbench.6.1.0.0/1010_revisit_I01_MOD_dat.xyz
../examples-secret/newclean/aarhus_workbench.6.1.0.0/1010_revisit_pronode_AVG_export.xyz
../examples-secret/newclean/aarhus_workbench.6.1.0.0/1010_revisit_I01_MOD_inv.xyz
../examples-secret/newclean/aarhus_workbench.6.1.0.0/1010_revisit_I01_MOD_syn.xyz
../exampl

In [34]:
import re

In [56]:
o = []
last = 0
for m in re.finditer(r"(\s|^)(?P<number>[0-9+-]\.[0-9E+-]+)($|\s)", d):
    s = m.start("number")
    o.append(d[last:s])
    o.append(('%.3E' % (np.random.random() * 2 * float(m["number"]))).upper())
    last = m.end("number")
o.append(d[last:])
o = "".join(o)

In [57]:
print(o)

/INFO
/Aarhus Workbench export file. File created: 07-10-2022 15:16:02. Exported from Workbench64. User: emeraldgeomodelling
/WB VERSION
/6.6.0.2
/WORKSPACE NAME
/G:\Test - 1\
/NODE NAME(S)
/NVE
/DUMMY
/9999
/DATA UNIT
/dB/dt [V/Am^4]
/DATA TYPE
/DTSKYTEM2
/COORDINATE SYSTEM
/WGS 84 UTM zone 32N (epsg:32632)
/NUMBER OF GATES
/Number of gates for channel 1 is 24
/Number of gates for channel 2 is 32
/GATE TIMES (s)
/Gates for channel 1 : 1.788E-07 4.350E-07 3.707E-06 4.435E-06 3.326E-06 8.440E-06 6.068E-06 1.294E-05 2.167E-05 2.093E-05 3.096E-05 3.344E-05 9.958E-06 5.393E-05 1.043E-04 8.643E-05 6.576E-05 1.369E-04 3.436E-04 2.179E-04 2.707E-04 3.469E-04 3.339E-04 5.519E-04 
/Gates for channel 2 : 3.030E-05 3.044E-05 6.328E-05 3.443E-05 4.666E-05 3.844E-05 3.012E-05 4.294E-05 3.847E-05 5.093E-05 1.014E-04 6.344E-05 9.022E-05 8.393E-05 5.310E-05 1.164E-04 1.155E-04 1.669E-04 9.882E-05 2.479E-04 1.841E-04 3.769E-04 6.550E-04 5.819E-04 1.294E-03 9.074E-04 1.502E-04 1.424E-03 1.217E-03 2.247E

In [50]:
print(d)

/INFO
/Aarhus Workbench export file. File created: 07-10-2022 15:16:02. Exported from Workbench64. User: emeraldgeomodelling
/WB VERSION
/6.6.0.2
/WORKSPACE NAME
/G:\Test - 1\
/NODE NAME(S)
/NVE
/DUMMY
/9999
/DATA UNIT
/dB/dt [V/Am^4]
/DATA TYPE
/DTSKYTEM2
/COORDINATE SYSTEM
/WGS 84 UTM zone 32N (epsg:32632)
/NUMBER OF GATES
/Number of gates for channel 1 is 24
/Number of gates for channel 2 is 32
/GATE TIMES (s)
/Gates for channel 1 : 1.000E-07 4.350E-07 2.435E-06 4.435E-06 6.435E-06 8.440E-06 1.043E-05 1.294E-05 1.643E-05 2.093E-05 2.643E-05 3.344E-05 4.243E-05 5.393E-05 6.843E-05 8.643E-05 1.089E-04 1.369E-04 1.724E-04 2.179E-04 2.749E-04 3.469E-04 4.379E-04 5.519E-04 
/Gates for channel 2 : 2.894E-05 3.044E-05 3.243E-05 3.443E-05 3.643E-05 3.844E-05 4.043E-05 4.294E-05 4.643E-05 5.093E-05 5.643E-05 6.344E-05 7.243E-05 8.393E-05 9.843E-05 1.164E-04 1.389E-04 1.669E-04 2.024E-04 2.479E-04 3.049E-04 3.769E-04 4.679E-04 5.819E-04 7.259E-04 9.074E-04 1.136E-03 1.424E-03 1.788E-03 2.247E

In [ ]:
"{:.2f}".format(50)

In [46]:
import numpy as np

str(np.random.random() * 2 * float("4.043E-05")).upper()

'2.6401561855996726E-05'

In [43]:
10**np.log10(float("4.043E-05"))

4.042999999999998e-05

In [54]:
'%.4E' % 0.000235678

'2.3568E-04'

In [20]:
re.match(r"(^|\s)[0-9.E+-]+", "1.2E4")

<re.Match object; span=(0, 5), match='1.2E4'>

In [24]:
re.findall("[0-9]+", "foo 123, 45 xx 78")

['123', '45', '78']

In [64]:
basepath = "../tests/data/aarhus_workbench.6.7.0.0"
for pathxyz in os.listdir(basepath):
    if not pathxyz.endswith("xyz"): continue
    pathxyz = os.path.join(basepath, pathxyz)
    print(pathxyz)
    data=libaarhusxyz.XYZ(pathxyz)
    pathdown = os.path.splitext(pathxyz)[0] + ".down"
    downfile.dump(data.model_dict, pathdown)
    
#parsed = libaarhusxyz.XYZ("../tests/data/aarhus_workbench.6.6.0.2/AVG_export_example_averagde_data_export.xyz").model_dict
#downfile.dump(parsed, "../tests/data/aarhus_workbench.6.6.0.2/AVG_export_example_averagde_data_export.down")

../tests/data/aarhus_workbench.6.6.0.2/SCI_1_Pro3_MOD_syn_example_SCI_inversion_export.xyz
../tests/data/aarhus_workbench.6.6.0.2/RAW_export_example_averagde_data_export.xyz
../tests/data/aarhus_workbench.6.6.0.2/SCI_1_Pro3_MOD_inv_example_SCI_inversion_export.xyz
../tests/data/aarhus_workbench.6.6.0.2/SCI_1_Pro3_MOD_dat_example_SCI_inversion_export.xyz
../tests/data/aarhus_workbench.6.6.0.2/AVG_export_example_averagde_data_export.xyz
